# ДЗ 5. Part-of-Speech разметка, NER, извлечение отношений

## Библиотеки

In [1]:
import pandas as pd
import re
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

import warnings
warnings.filterwarnings("ignore")

## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


### Загрузка данных

In [2]:
full_train = pyconll.load_from_file('data/pyconll/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('data/pyconll/ru_syntagrus-ud-dev.conllu')

In [3]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [4]:
# Создадим датафрейм для сравнения теггеров
info = pd.DataFrame(columns=['accuracy'])

In [5]:
# Заполним данными
df_train = []
for sent in full_train[:]:
    df_train.append([(token.form, token.upos) for token in sent])
    
df_test = []
for sent in full_test[:]:
    df_test.append([(token.form, token.upos) for token in sent])
    
df_sent = []
for sent in full_test[:]:
    df_sent.append([token.form for token in sent])

**Проверим UnigramTagger, BigramTagger, TrigramTagger и их комбинации**

In [7]:
taggers = [UnigramTagger, BigramTagger, TrigramTagger]

for tagg in taggers:
    tagger = tagg(df_train)
    name_tag = ''.join([ch for ch in list(re.split('\.', str(tagg))[-1] ) if ch not in string.punctuation])
    info.loc[name_tag, 'accuracy'] = tagger.evaluate(df_test)

In [8]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

In [9]:
backoff = DefaultTagger('NOUN') 
tagg = backoff_tagger(df_train, taggers, backoff = backoff)

info.loc['BackoffTagger', 'accuracy'] = tagg.evaluate(df_test)

In [10]:
info

,accuracy
UnigramTagger,0.877254
BigramTagger,0.696306
TrigramTagger,0.248087
BackoffTagger,0.911999


**Напишем свой теггер, как на занятии, попробуем разные векторайзеры, добавим знание не только букв, но и слов**

In [13]:
# Теггер

train_token = []
train_label = []
for sent in df_train[:]:
    for token in sent:
        train_token.append(token[0])
        train_label.append('NO_TAG' if token[1] is None else token[1])
        
test_token = []
test_label = []
for sent in df_test[:]:
    for token in sent:
        test_token.append(token[0])
        test_label.append('NO_TAG' if token[1] is None else token[1])

In [14]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_label)
test_labels = encoder.transform(test_label)
encoder.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [15]:
# Векторайзеры

vectorizers = [CountVectorizer, HashingVectorizer, TfidfVectorizer]

for vect in vectorizers:

    vectorizer = vect(ngram_range=(1, 5), analyzer='char')
    X_train = vectorizer.fit_transform(train_token)
    X_test = vectorizer.transform(test_token)

    lr = LogisticRegression(random_state=0)
    lr.fit(X_train, train_labels)

    pred = lr.predict(X_test)
    name_vect = ''.join([ch for ch in list(re.split('\.', str(vect))[-1] ) if ch not in string.punctuation])
    info.loc[name_vect, 'accuracy'] = accuracy_score(test_labels, pred)

**Сравниваем все полученные результаты**

In [16]:
info

,accuracy
UnigramTagger,0.877254
BigramTagger,0.696306
TrigramTagger,0.248087
BackoffTagger,0.911999
CountVectorizer,0.94998
HashingVectorizer,0.93024
TfidfVectorizer,0.938227


# Задание 2

С установкой библиотек слишком много сложностей...